In [ ]:
!pip install -q transformers datasets

Description : Cette ligne de code utilise la commande pip pour installer les bibliothèques transformers et datasets. Ces bibliothèques sont des outils essentiels pour travailler avec des modèles de Transformers pré-entraînés et des ensembles de données dans le domaine du Traitement Automatique du Langage Naturel (NLP).

In [ ]:
!pip install -q pytorch-lightning wandb

Description : Cette ligne de code utilise la commande pip pour installer silencieusement les bibliothèques pytorch-lightning et wandb. Ces bibliothèques sont utilisées pour faciliter l'entraînement de modèles PyTorch et pour effectuer un suivi et une visualisation avancés des expériences d'entraînement.

In [ ]:
# Importing the necessary libraries and modules

from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, get_linear_schedule_with_warmup
from torch.optim import AdamW
import torch
import pytorch_lightning as pl
from tqdm import tnrange
from collections import Counter
from sklearn.model_selection import train_test_split

# from evaluate import load
# import bert_score
from transformers import logging
logging.set_verbosity_error()
import matplotlib.pyplot as plt

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Description :
Ce code commence par importer les bibliothèques et modules nécessaires pour le projet. Ces bibliothèques incluent des outils pour la gestion des données, la création de modèles, l'entraînement, la visualisation, et plus encore. La désactivation de la parallélisation des tokenizers est également effectuée pour améliorer les performances.

La configuration `os.environ["TOKENIZERS_PARALLELISM"] = "false"` a pour but de désactiver la parallélisation des tokenizers dans la bibliothèque Transformers. Par défaut, Transformers essaie d'accélérer le prétraitement des données en parallélisant les opérations de tokenization sur plusieurs cœurs de processeur (si disponibles). Cela peut être utile pour accélérer le traitement des données, en particulier lorsqu'il s'agit de grands jeux de données.

Cependant, dans certains environnements ou configurations, la parallélisation automatique peut entraîner des problèmes, tels que des conflits de ressources ou des erreurs liées à la gestion des threads. Pour éviter ces problèmes, cette configuration désactive la parallélisation, ce qui peut ralentir légèrement le prétraitement des données, mais assure une exécution plus stable du code, en particulier sur des systèmes avec des contraintes de ressources.

In [ ]:
# load the dataset

dataset = load_dataset("Owishiboo/grammar-correction", split="train")
print(dataset)

Ce code charge un ensemble de données provenant du jeu de données "Owishiboo/grammar-correction" en utilisant la bibliothèque Hugging Face datasets. Plus précisément, il charge la partition d'entraînement (split="train") de ce jeu de données.

In [ ]:
# Separation of the dataset into training, validation and test sets

train_testvalid = dataset.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

dataset

Cette séquence de code divise le dataset en trois parties distinctes : l'ensemble d'entraînement, l'ensemble de test et l'ensemble de validation. Cela garantit que nous disposons de données distinctes pour chaque phase de notre modèle de correction orthographique en NLP.

80 % pour l'entraînement, 10 % pour le test et 10 % pour la validation.

Nous organisons finalement nos données divisées dans une structure de type DatasetDict qui contient trois sous-ensembles : train, test et validation.


In [ ]:
# Displaying a sample data example

example = dataset['train'][0]

print("input:", example["input"])
print("target:", example["target"])

Cela nous permet de visualiser un exemple spécifique que notre modèle de correction orthographique tentera de corriger pendant l'entraînement.

In [ ]:
# Tokenizer Configuration and Preprocessing Function

tokenizer = AutoTokenizer.from_pretrained("t5-small")

prefix = "correction: "
max_input_length = 600
max_target_length = 600

def preprocessor(examples, prefix='correction:', max_input_length=512, max_target_length=512):
  sentences = examples['input']
  corrections = examples['target']

  # encode the input sentences
  inputs = [prefix + sentence for sentence in sentences]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the target sentences and extract input_ids
  labels = tokenizer(corrections, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # Replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)

  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Ce code configure le tokenizer pour le modèle T5-small et définit une fonction de prétraitement preprocessor qui prend des exemples d'entrée et de cible, ajoute un préfixe au texte d'entrée, les tokenise, gère les longueurs maximales, et formate les étiquettes avec des indices -100 pour ignorer les tokens de rembourrage lors du calcul de la perte. Cette fonction de prétraitement est utilisée pour formater les données d'entraînement du modèle.

In [ ]:
# Dataset Mapping

# Apply the preprocessing function to the entire dataset in batches
dataset = dataset.map(preprocessor, batched=True)
print(dataset)


Ce code prend le jeu de données initial et applique la fonction de prétraitement preprocessor à l'ensemble du jeu de données en traitant les données par lots. Ensuite, il affiche le jeu de données modifié, qui est maintenant formaté et prêt à être utilisé pour l'entraînement du modèle.

**"input_ids" (identifiants de tokens d'entrée) :**

Il s'agit d'une liste (ou d'un tableau) d'identifiants numériques qui représentent chaque token (mot ou sous-mot) dans le texte d'entrée. Ces identifiants sont généralement des entiers qui correspondent à des indices dans un vocabulaire préalablement défini. Chaque mot ou sous-mot est converti en un identifiant unique. Cela permet au modèle de comprendre le texte d'entrée sous une forme numérique.

**"attention_mask" (masque d'attention) :**

Le masque d'attention est une autre liste (ou tableau) de valeurs binaires (0 ou 1) qui a la même longueur que les "input_ids". Il indique au modèle quels tokens sont importants pour la tâche à accomplir (1 pour les tokens pertinents, 0 pour les tokens non pertinents). Il est utilisé pour spécifier où le modèle doit porter son attention et où il peut ignorer certains tokens. Cela aide à gérer les séquences de longueur variable.

**"labels" (étiquettes ou cibles) :**

Les "labels" sont généralement utilisés dans des tâches d'apprentissage supervisé. Ils représentent la sortie attendue du modèle pour un exemple donné. Par exemple, dans une tâche de classification de texte, les "labels" indiqueraient la classe ou la catégorie à laquelle appartient l'exemple d'entrée. Le modèle apprend à faire des prédictions en se basant sur ces étiquettes lors de l'entraînement.

Ces trois éléments sont essentiels pour préparer et alimenter des données dans un modèle NLP, car ils permettent de représenter le texte d'entrée, de guider l'attention du modèle et de spécifier les sorties attendues pour l'entraînement.

In [ ]:
# Dataset Formatting

# Set the dataset format to "torch"
dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
print(dataset)


Ce code configure le format du jeu de données pour qu'il soit compatible avec PyTorch en spécifiant les colonnes à inclure. Cela signifie que le jeu de données est maintenant prêt à être utilisé avec des modèles PyTorch, car il est dans un format approprié pour l'entraînement et l'évaluation.

In [ ]:
# Data Loaders Configuration

# Determine the number of CPU cores available.
num_workers = os.cpu_count()

# Create data loaders for training, validation, and testing datasets.
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8, num_workers=num_workers)
valid_dataloader = DataLoader(dataset['valid'], batch_size=4, num_workers=num_workers)
test_dataloader = DataLoader(dataset['test'], batch_size=4, num_workers=num_workers)


Ce code configure les chargeurs de données pour les ensembles d'entraînement, de validation et de test. Il utilise le nombre de cœurs de CPU disponibles pour spécifier le nombre de travailleurs (num_workers) à utiliser lors du chargement des données, ce qui permet un chargement plus rapide. Les données d'entraînement sont mélangées (shuffle=True) pour améliorer la généralisation du modèle lors de l'entraînement.

In [ ]:
# Fetch a batch from the training data loader

batch = next(iter(train_dataloader))
print(batch.keys())


Ce code extrait un lot de données du chargeur de données d'entraînement (train_dataloader) en utilisant la fonction next(iter(train_dataloader)). Ensuite, il affiche les clés du lot (batch.keys()) pour montrer quelles sont les différentes composantes de données disponibles dans le lot, telles que les identifiants d'entrée (input_ids), les masques d'attention (attention_mask), et les étiquettes (labels). Ces composantes sont essentielles pour former le modèle et effectuer des calculs d'entraînement.

In [ ]:
# Decode the input_ids of the first example in the batch.
tokenizer.decode(batch['input_ids'][0])

In [ ]:
# Decode the labels of the first example in the batch, filtering out -100 tokens
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

Les étiquettes contiennent les identifiants des mots corrigés pour le premier exemple. Ensuite, il décode ces étiquettes en texte en utilisant le tokenizer. Cependant, il filtre les tokens ayant la valeur -100, car ce sont les tokens de rembourrage et ne sont pas nécessaires pour l'affichage du texte corrigé. Le texte décodé est ensuite imprimé pour afficher la correction orthographique du modèle pour cet exemple.

In [ ]:
class T5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        # Initializes the T5 model and sets up training-related attributes
        super().__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
        self.train_losses=[]
        self.validation_losses=[]
        self.config=self.model.config

        self.train_losses_epoch=[]
        self.validation_losses_epoch=[]

        self.save_hyperparameters()


    def forward(self, input_ids, attention_mask, labels=None):
        # Performs a forward pass through the T5 model
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs


    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss
        return loss


    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)
        self.train_losses.append(loss)
        return loss


    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("validation_loss", loss, on_epoch=True)
        self.validation_losses.append(loss)
        return loss


    def on_train_epoch_end(self):
        # Calculate average loss for the epoch and append to the list
        avg_train_loss = sum(self.train_losses)/ len(self.train_losses)
        self.train_losses_epoch.append(avg_train_loss.item())

        # Reset epoch loss accumulator
        self.train_losses = []


    def on_validation_epoch_end(self):
        # Calculate average loss for the epoch and append to the list
        avg_val_loss = sum(self.validation_losses) / len(self.validation_losses)
        self.validation_losses_epoch.append(avg_val_loss.item())

        # Reset epoch loss accumulator
        self.validation_losses = []

        # Reset epoch loss accumulator
        self.test_losses = []


    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.model.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}


    def generate(self, input_ids, max_new_tokens=100, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        input_ids=input_ids.clone().detach().reshape((1,-1)).to(device)
        return self.model.generate(input_ids, max_new_tokens=max_new_tokens)


    def push_to_hub(self, model_name, organization):
        # Save the model
        self.model.push_to_hub(model_name, organization)


    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

Description : Cette classe T5 définit un modèle Lightning pour l'entraînement et l'évaluation d'un modèle T5-small pour la correction orthographique. Le modèle est configuré pour effectuer des étapes d'entraînement, de validation et de test, enregistrant les pertes et utilisant un planificateur de taux d'apprentissage. Elle offre également la possibilité de générer du texte et de pousser le modèle vers un référentiel de modèle (Hub).

In [ ]:
# Hyperparameters
lr = 5e-5
num_train_epochs = 10
warmup_steps = 1000
patience = 2
max_epochs = 10

# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
# Early stopping callback - Stops training if validation loss doesn't improve for 'patience' epochs
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=patience,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')
# Model checkpoint callback - Saves the best model during training based on validation loss
checkpoint_callback = ModelCheckpoint(dirpath='./', monitor='validation_loss', mode='min', save_top_k = 1)

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
# Create a PyTorch Lightning Trainer instance with defined callbacks and max_epochs
trainer = Trainer(accelerator=accelerator,
                  callbacks=[early_stop_callback, lr_monitor, checkpoint_callback], max_epochs=max_epochs)

# Create an instance of our T5 model with specified hyperparameters
model = T5(lr=lr, num_train_epochs=num_train_epochs, warmup_steps=warmup_steps)
# Train the model
trainer.fit(model)
tokenizer.save_pretrained("./")
checkpoint_path = checkpoint_callback.best_model_path

Ce code configure les hyperparamètres pour l'entraînement, définit les rappels pour l'arrêt anticipé, la surveillance du taux d'apprentissage et l'enregistrement du modèle, puis entraîne le modèle T5 à l'aide du formateur PyTorch Lightning. Il enregistre également les poids du tokenizer et récupère le chemin du meilleur point de contrôle du modèle.

In [ ]:
# Retrieve the training and validation losses of the model

train_losses=model.train_losses_epoch
validation_losses=model.validation_losses_epoch


Ce code récupère les pertes d'entraînement et de validation d'un modèle. Les pertes d'entraînement sont les pertes calculées lors de l'entraînement du modèle sur le jeu de données d'entraînement, tandis que les pertes de validation sont les pertes calculées lors de la validation du modèle sur un jeu de données de validation distinct. Ces pertes sont généralement utilisées pour évaluer les performances du modèle et surveiller son apprentissage au fil des époques.

In [ ]:
print('Loss on validation set before fine tuning: ', validation_losses[0])

In [ ]:
# Plotting the losses
plt.plot(train_losses, label='Training Loss')
plt.plot(validation_losses[1:], label='Validation Loss')

# Adding labels and title
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('T5: Losses over Epochs')

# Adding legend
plt.legend()

plt.savefig('losses_plot.png')
# Displaying the plot
plt.show()

Ce code génère un graphique représentant l'évolution des pertes (losses) d'un modèle T5 au fil des époques d'entraînement. Les pertes d'entraînement sont tracées en bleu, tandis que les pertes de validation sont tracées en orange (à l'exception de la première valeur, car elle peut être aberrante en raison de la phase d'initialisation). Ce graphique permet de visualiser comment les pertes du modèle changent à mesure qu'il s'entraîne et de surveiller son apprentissage. Le graphique est ensuite sauvegardé en tant qu'image sous le nom 'losses_plot.png'.

In [ ]:
data_id = 9
test_dataset = dataset['test']
test_input = test_dataset['input'][data_id]
test_input_ids = test_dataset[data_id]['input_ids']
test_target = test_dataset['target'][data_id]
test_attention_mask = test_dataset[data_id]['attention_mask']

Ce code permet de sélectionner un échantillon de données spécifique dans l'ensemble de données de test.
Les informations  de cet echantillon seront utilisées pour évaluer le modèle sur cet échantillon spécifique.

In [ ]:
# Checking if a GPU is available, and setting the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Moving the model to the selected device (GPU or CPU)
model = model.to(device)


In [ ]:
print("Input sentence:")
print(test_input)

In [ ]:
print("target:")
print(test_target)

In [ ]:
# Generating corrected text using the fine-tuned T5 model
output_ids = model.generate(test_input_ids.reshape((1,-1)))

# Decoding the generated output and removing special tokens
corrected_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)


Ce code génère une phrase corrigée en utilisant le modèle T5 fine-tuné. Il prend l'ID d'entrée de la phrase test, le passe au modèle, récupère la sortie générée, puis décode cette sortie en une phrase corrigée en supprimant les jetons spéciaux ajoutés par le modèle. En fin de compte, corrected_sentence contient la phrase corrigée.

In [ ]:
print("Corrected sentence:")
print(corrected_sentence)